# 1. Libraries

In [23]:
# Import Libraries
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# 2. Introduction

The purpose of this notebook is to gather data on the European ARMS Challenge (EUAC) tournament series. <br>
The EUAC is an online biweekly tournament series for the video game "ARMS" released for the Nintendo Switch console in 2017. <br>
The EUAC started in 2017 and ran consistently until 2022.<br>
<br>
More information can be found in references below.

This notebook is to purely gather the data, act as a proof of concept, and format it for future use/analysis. <br>
It will then be exported to a csv file if successful. <br>

# 3. Objectives

- Determine what data that is necessary
- Gather EUAC#1 data from start.gg using their api
- Gather EUAC#2 to EUAC#110 data from Challonge.com using their api
- Format the data into a pandas dataframe
- Export to csv

## 3.1. Objective 1

### Determine what data is necessary

For each tournament, we will need: <br>
- The participants who entered
- Each match that was played
- The result of each match
- The date the tournament took place
- The final placements
- Each player's seeding

## 3.2. Objective 2 - Data Gathering

The tournaments were mostly hosted on Challonge.com but the first one was hosted on start.gg. <br>

### 3.2.1. - Start.gg

Start.gg's API uses GraphQL as its query language <br>
The GraphQL queries will be wrapped in multiline strings so they can be sent to the start.gg API to retrieve the data. <br>
Queries also expect an API token to passed through the header. This token will be loaded from a .env file.<br>
<br>
A link to the tutorial that was followed to come up with the queries can be found in References.

In [24]:
# Load API Keys stored in .env file
load_dotenv(".env")
START_GG_API_TOKEN = os.getenv("START_GG_API_TOKEN")

The tournament was hosted at this url: https://www.start.gg/tournament/eu-arms-challenge-1/events <br>
For some of the information that is needed, the id of the event is required. Which is obtained here using the url slug

In [25]:
# GraphQL query wrapped in a multiline string. 
query = """
query TournamentQuery {
  tournament(slug: "tournament/eu-arms-challenge-1") {
    name
    events {
      name
      id
    }
  }
}
"""

In [26]:
# Sends the query and returns the response. Will be using this a lot
def run_query(query):
    url = "https://api.start.gg/gql/alpha"
    headers = {
        "Authorization": f"Bearer {START_GG_API_TOKEN}",
        "Content-Type": "application/json"
    }
    response = requests.post(url, json={'query': query}, headers=headers)
    return response.json()

In [27]:
data = run_query(query)

In [28]:
data

{'data': {'tournament': {'name': 'EU ARMS Challenge #1',
   'events': [{'name': 'EU ARMS Challenge #1', 'id': 53835}]}},
 'extensions': {'cacheControl': {'version': 1,
   'hints': [{'path': ['tournament'], 'maxAge': 300, 'scope': 'PRIVATE'}]},
  'queryComplexity': 2},
 'actionRecords': []}

In [29]:
eventId = data["data"]["tournament"]["events"][0]["id"]

In [30]:
eventId

53835

The id of the event is 53835

Using the event id, a query can be written to get the players, their seedings and placements, and the start date of the tournament <br>
Some of this information could have been gotten before but not all

In [31]:
# Query for players, seedings, placements

query = """
query {
  event(id: 53835) {
    id
    name
    startAt
    standings(query: {
      page: 1
    }) {
      nodes {
        placement
        entrant {
          id
          name
          seeds {
            seedNum
          }
        }
      }
    }
  }
}
"""

In [32]:
data = run_query(query)

In [33]:
data

{'data': {'event': {'id': 53835,
   'name': 'EU ARMS Challenge #1',
   'startAt': 1508605200,
   'standings': {'nodes': [{'placement': 1,
      'entrant': {'id': 1152276,
       'name': 'FR | Maxou0708',
       'seeds': [{'seedNum': 20}]}},
     {'placement': 2,
      'entrant': {'id': 1118825,
       'name': 'Rapha_MTH',
       'seeds': [{'seedNum': 6}]}},
     {'placement': 3,
      'entrant': {'id': 1133810, 'name': 'Sabaca', 'seeds': [{'seedNum': 8}]}},
     {'placement': 4,
      'entrant': {'id': 1152521,
       'name': 'TCM | Raffa',
       'seeds': [{'seedNum': 21}]}},
     {'placement': 5,
      'entrant': {'id': 1152258,
       'name': 'FrankTank',
       'seeds': [{'seedNum': 19}]}},
     {'placement': 5,
      'entrant': {'id': 1114568, 'name': 'ocrim', 'seeds': [{'seedNum': 2}]}},
     {'placement': 7,
      'entrant': {'id': 1139232,
       'name': 'Alumento',
       'seeds': [{'seedNum': 11}]}},
     {'placement': 7,
      'entrant': {'id': 1133727,
       'name': 'SC☆Mo

Data is a nested dictionary. Now we'll gather a list of participants, their placements and seeds, by digging through this

In [34]:
data["data"]["event"]["name"]

'EU ARMS Challenge #1'

In [35]:
data["data"]["event"]["startAt"]

1508605200

In [36]:
tournamentDate = data["data"]["event"]["startAt"]

UNIX time for when tournament took place

In [37]:
data["data"]["event"]["standings"]

{'nodes': [{'placement': 1,
   'entrant': {'id': 1152276,
    'name': 'FR | Maxou0708',
    'seeds': [{'seedNum': 20}]}},
  {'placement': 2,
   'entrant': {'id': 1118825, 'name': 'Rapha_MTH', 'seeds': [{'seedNum': 6}]}},
  {'placement': 3,
   'entrant': {'id': 1133810, 'name': 'Sabaca', 'seeds': [{'seedNum': 8}]}},
  {'placement': 4,
   'entrant': {'id': 1152521,
    'name': 'TCM | Raffa',
    'seeds': [{'seedNum': 21}]}},
  {'placement': 5,
   'entrant': {'id': 1152258,
    'name': 'FrankTank',
    'seeds': [{'seedNum': 19}]}},
  {'placement': 5,
   'entrant': {'id': 1114568, 'name': 'ocrim', 'seeds': [{'seedNum': 2}]}},
  {'placement': 7,
   'entrant': {'id': 1139232, 'name': 'Alumento', 'seeds': [{'seedNum': 11}]}},
  {'placement': 7,
   'entrant': {'id': 1133727, 'name': 'SC☆Momso', 'seeds': [{'seedNum': 7}]}},
  {'placement': 9,
   'entrant': {'id': 1114889,
    'name': 'VilleViljar',
    'seeds': [{'seedNum': 4}]}},
  {'placement': 9,
   'entrant': {'id': 1152242,
    'name': 'TC

In [38]:
data["data"]["event"]["standings"]["nodes"][0] #Information for one player

{'placement': 1,
 'entrant': {'id': 1152276,
  'name': 'FR | Maxou0708',
  'seeds': [{'seedNum': 20}]}}

In [39]:
# Placement
data["data"]["event"]["standings"]["nodes"][0]["placement"]

1

In [40]:
# Name
data["data"]["event"]["standings"]["nodes"][0]["entrant"]["name"]

'FR | Maxou0708'

In [41]:
# id
data["data"]["event"]["standings"]["nodes"][0]["entrant"]["id"]

1152276

In [42]:
# Seeding
data["data"]["event"]["standings"]["nodes"][0]["entrant"]["seeds"][0]["seedNum"]

20

In [43]:
# Putting it all together in a for loop. Store information in arrays

playerArray = []
placementArray = []
seedArray = []
idArray = []

for entrant in data["data"]["event"]["standings"]["nodes"]:
    playerArray.append(entrant["entrant"]["name"])
    idArray.append(entrant["entrant"]["id"])
    placementArray.append(entrant["placement"])
    seedArray.append(entrant["entrant"]["seeds"][0]["seedNum"])

In [44]:
# Make a pandas dataframe of the arrays

playerdf = pd.DataFrame({
    "Start ID": idArray,
    "Player": playerArray,
    "Seed": seedArray,
    "Placement": placementArray
})

In [45]:
playerdf.head()

,Start ID,Player,Seed,Placement
0,1152276,FR | Maxou0708,20,1
1,1118825,Rapha_MTH,6,2
2,1133810,Sabaca,8,3
3,1152521,TCM | Raffa,21,4
4,1152258,FrankTank,19,5


All players, their Start ids, their placements and seeding, have been gathered. <br>To note for later: players can sign up with "tags". Denoted with a |. But | can be present in a tag

Due to the hierarchical structure of how tournaments in Start.gg can be, we need to use the event ID to get the phase ID, use the Phase Id to get the Phase Group ID, and then using the Phase Group ID we can obtain the sets in the tournaments that were played with their reported scores

In [46]:
# Query to get Phase ID

query = """
query {
  event(id: 53835) {
    id
    name
    phases {
      id
      name
    }
  }
}"""

In [47]:
data = run_query(query)

In [48]:
data

{'data': {'event': {'id': 53835,
   'name': 'EU ARMS Challenge #1',
   'phases': [{'id': 159661, 'name': 'Bracket'}]}},
 'extensions': {'cacheControl': {'version': 1,
   'hints': [{'path': ['event'], 'maxAge': 60, 'scope': 'PRIVATE'}]},
  'queryComplexity': 2},
 'actionRecords': []}

In [49]:
# Phase ID
data["data"]["event"]["phases"][0]["id"]

159661

In [50]:
# Query to get Phase Group ID
query = """
query {
  phase(id: 159661) {
    phaseGroups {
      nodes {
        id
      }
    }
  }
}"""

In [51]:
data = run_query(query)

In [52]:
data

{'data': {'phase': {'phaseGroups': {'nodes': [{'id': 431370}]}}},
 'extensions': {'cacheControl': {'version': 1, 'hints': None},
  'queryComplexity': 1},
 'actionRecords': []}

In [53]:
# Phase Group ID
data["data"]["phase"]["phaseGroups"]["nodes"][0]["id"]

431370

In [54]:
# Query to get all sets

query = """
query {
  phaseGroup(id: 431370) {
    sets(page: 1, perPage: 100) {
      nodes {
        id
        displayScore
        fullRoundText
        winnerId
        slots {
          entrant {
            name
          }
        }
      }
    }
  }
}"""

In [55]:
data = run_query(query)

In [56]:
# Display all sets
data

{'data': {'phaseGroup': {'sets': {'nodes': [{'id': 10598232,
      'displayScore': 'Rapha_MTH 1 - FR | Maxou0708 3',
      'fullRoundText': 'Grand Final',
      'winnerId': 1152276,
      'slots': [{'entrant': {'name': 'Rapha_MTH'}},
       {'entrant': {'name': 'FR | Maxou0708'}}]},
     {'id': 10598233,
      'displayScore': 'FR | Maxou0708 3 - Rapha_MTH 0',
      'fullRoundText': 'Grand Final Reset',
      'winnerId': 1152276,
      'slots': [{'entrant': {'name': 'FR | Maxou0708'}},
       {'entrant': {'name': 'Rapha_MTH'}}]},
     {'id': 10598231,
      'displayScore': 'Sabaca 0 - Rapha_MTH 2',
      'fullRoundText': 'Winners Final',
      'winnerId': 1118825,
      'slots': [{'entrant': {'name': 'Sabaca'}},
       {'entrant': {'name': 'Rapha_MTH'}}]},
     {'id': 10598295,
      'displayScore': 'DQ',
      'fullRoundText': 'Losers Final',
      'winnerId': 1152276,
      'slots': [{'entrant': {'name': 'Sabaca'}},
       {'entrant': {'name': 'FR | Maxou0708'}}]},
     {'id': 1059829

In [68]:
data["data"]["phaseGroup"]["sets"]["nodes"][0]

{'id': 10598232,
 'displayScore': 'Rapha_MTH 1 - FR | Maxou0708 3',
 'fullRoundText': 'Grand Final',
 'winnerId': 1152276,
 'slots': [{'entrant': {'name': 'Rapha_MTH'}},
  {'entrant': {'name': 'FR | Maxou0708'}}]}

In [67]:
data["data"]["phaseGroup"]["sets"]["nodes"][-3]

{'id': 10598210,
 'displayScore': 'DQ',
 'fullRoundText': 'Winners Round 1',
 'winnerId': 1152122,
 'slots': [{'entrant': {'name': 'Kotorious BRD'}},
  {'entrant': {'name': 'Altair'}}]}

In [58]:
data["data"]["phaseGroup"]["sets"]["nodes"][7]["displayScore"]

'ocrim 0 - FR | Maxou0708 2'

Player names and Scores can be gotten from "displayScore" but DQs will require a bit more work. <br>
Will have to access "slots" to get player names, query dataframe for the winner id. Loser is the other.

In [59]:
import re

In [60]:
scoreSplit = data["data"]["phaseGroup"]["sets"]["nodes"][7]["displayScore"].split(" - ")

In [61]:
scoreSplit

['ocrim 0', 'FR | Maxou0708 2']

In [62]:
scoreSplit[0]

'ocrim 0'

In [63]:
scoreSplit[1][-1]

'2'

In [64]:
if int(scoreSplit[0][-1]) > int(scoreSplit[1][-1]):
    score = f"{scoreSplit[0][-1]}-{scoreSplit[1][-1]}"
elif int(scoreSplit[1][-1]) > int(scoreSplit[0][-1]):
    score = f"{scoreSplit[1][-1]}-{scoreSplit[0][-1]}"
else:
    score = "0--1" #Disqualification

In [65]:
score

'2-0'

### 3.2.2 - Challonge.com

# References

- https://armswiki.org/wiki/EU_ARMS_Challenge
- https://developer.start.gg/docs/examples/queries/get-event